<a href="https://colab.research.google.com/github/iam-Dylan/automated-essay-scoring/blob/dylan/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XÂY DỰNG MÔ HÌNH

## Import các thư viện cần thiết

In [1]:
#!pip install pyspellchecker

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

import pandas as pd
import polars as pl
import numpy as np
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score, accuracy_score
import lightgbm as lgb

import nltk
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
from spellchecker import SpellChecker
import warnings
warnings.filterwarnings('ignore')

Mounted at /gdrive


## Đọc file dữ liệu

In [3]:
train = pd.read_csv('/gdrive/MyDrive/PTDLTM - Project /Notebook/learning-agency-lab-automated-essay-scoring-2/train.csv')
train

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3
...,...,...,...
17302,ffd378d,"the story "" The Challenge of Exploing Venus "" ...",2
17303,ffddf1f,Technology has changed a lot of ways that we l...,4
17304,fff016d,If you don't like sitting around all day than ...,2
17305,fffb49b,"In ""The Challenge of Exporing Venus,"" the auth...",1


In [4]:
test = pd.read_csv('/gdrive/MyDrive/PTDLTM - Project /Notebook/learning-agency-lab-automated-essay-scoring-2/test.csv')
test

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


## Tiền xử lý dữ liệu

### Làm sạch văn bản

Cần làm sạch văn bản, nhằm chuẩn hóa và loại bỏ những thành phần không cần thiết trước khi tiến hành các bước xử lý tiếp theo.
- Văn bản được chuyển đổi toàn bộ về chữ thường để đảm bảo tính nhất quán và tránh phân biệt giữa chữ hoa và chữ thường.
- Các thẻ HTML, thẻ tên người dùng (bắt đầu bằng `@`), hashtag (bắt đầu bằng `#`), và đường dẫn URL đều được loại bỏ để giữ lại nội dung văn bản thực sự. - Các ký tự đặc biệt, dấu câu và các số trong văn bản, thường không mang lại giá trị ngữ nghĩa, cũng được loại bỏ.
- Khoảng trắng dư thừa hay dấu câu liên tiếp được xử lý và thay thế bằng một ký tự duy nhất.
- Các từ viết tắt được mở rộng thành dạng đầy đủ để đảm bảo tính nhất quán.

In [5]:
def clean_text(text):
    # Chuyển chữ viết hoa thành chữ thường
    text = text.lower()

    # Xóa các thẻ HTML
    text = re.compile(r'<.*?>').sub(r'', text)

    # Xóa các tag tên (mention)
    text = re.sub(r'@\w+\s*', '', text)

    # Xóa hashtag (dấu #)
    text = re.sub(r'#\w+', '', text)

    # Xóa các liên kết URL
    text = re.sub(r'http\S+|www\S+', '', text)

    # Xóa các ký tự không mong muốn như \xa0
    text = text.replace(u'\xa0', ' ')

    # Xóa chữ số
    text = re.sub(r'\d+', '', text)

    # Thay thế các khoảng trắng liên tiếp bằng một khoảng trắng duy nhất
    text = re.sub(r'\s+', ' ', text)

    # Mở rộng các từ viết tắt
    # text = expandContractions(text)

    # Thay thế các dấu chấm và dấu phẩy liên tiếp bằng một dấu duy nhất
    text = re.sub(r'\.+', '.', text)
    text = re.sub(r'\,+', ',', text)

    # Xóa các khoảng trắng ở đầu và cuối chuỗi
    text = text.strip()

    return text

train['full_text'] = train['full_text'].apply(clean_text)
test['full_text'] = test['full_text'].apply(clean_text)

## Feature engineering

### Rút trích đặc trưng đoạn văn

Phân tích về đặc trưng đoạn văn giúp hiểu rõ hơn về cấu trúc văn bản, độ phức tạp và ngữ cảnh văn bản.
- Độ dài đoạn văn phản ánh mức độ chi tiết và độ phức tạp của văn bản.
- Số câu trong đoạn văn giúp xác định mức độ chi tiết và cách trình bày của văn bản.
- Số từ trong đoạn văn là một chỉ số quan trọng để đo lường độ phức tạp và độ dài của văn bản.
- Lấy giá trị đầu tiên và cuối cùng của mỗi đặc trưng  giúp hiểu thêm về sự biến đổi và xu hướng của các đoạn văn trong văn bản.
- Ngoài ra độ lệch chuẩn của các đặc trưng phản ánh mức độ phân tán và sự biến động của các đoạn văn.

In [6]:
def preprocess_paragraphs(df):
    df = df.explode('paragraph')
    df = df.filter(pl.col('paragraph').str.strip() != "")  # Loại bỏ các đoạn văn trống
    df = df.with_columns(pl.col('paragraph').map_elements(clean_text))
    df = df.with_columns(pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_length"))
    df = df.with_columns(
        pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias("sentence_count"),
        pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias("word_count")
    )
    return df

# Hàm tính toán các đặc trưng từ đoạn văn
def extract_paragraph_features(df):
    features = ['paragraph_length', 'sentence_count', 'word_count']
    aggs = [
        *[pl.col('paragraph_length').filter(pl.col('paragraph_length') >= i).count().alias(f"length_ge_{i}_count") for i in [50, 75, 100, 125, 150, 175, 200, 250, 300, 350, 400, 500, 600, 700]],
        *[pl.col('paragraph_length').filter(pl.col('paragraph_length') <= i).count().alias(f"length_le_{i}_count") for i in [25, 49]],
        *[pl.col(feat).max().alias(f"{feat}_max") for feat in features],
        *[pl.col(feat).mean().alias(f"{feat}_mean") for feat in features],
        *[pl.col(feat).min().alias(f"{feat}_min") for feat in features],
        *[pl.col(feat).first().alias(f"{feat}_first") for feat in features],
        *[pl.col(feat).last().alias(f"{feat}_last") for feat in features],
        *[pl.col(feat).sum().alias(f"{feat}_sum") for feat in features],
        *[pl.col(feat).kurtosis().alias(f"{feat}_kurtosis") for feat in features],
        *[pl.col(feat).quantile(0.25).alias(f"{feat}_q1") for feat in features],
        *[pl.col(feat).quantile(0.75).alias(f"{feat}_q3") for feat in features],
    ]
    df = df.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    return df.to_pandas()

# Chuyển đổi dữ liệu và xử lý các đoạn văn
def process_data(data):
    # Tách các đoạn văn bằng dấu "\n\n" và loại bỏ các đoạn trống
    columns = [pl.col("full_text").str.split(by="\n\n").alias("paragraph")]
    data = pl.from_pandas(data).with_columns(columns)
    return preprocess_paragraphs(data)

# Sử dụng các hàm đã cải tiến trên dữ liệu train và test
train = process_data(train)
test = process_data(test)

train_feats = extract_paragraph_features(train)
test_feats = extract_paragraph_features(test)

# Đếm số đặc trưng
feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], train_feats.columns))
print('Feature count in train set: ', len(feature_names))
feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], test_feats.columns))
print('Feature count in test set: ', len(feature_names))

Feature count in train set:  43
Feature count in test set:  43


### Rút trích đặc trưng câu văn

Tương tự, việc phân tích về đặc trưng câu văn cũng giúp hiểu rõ hơn về cấu trúc, độ phức tạp và ngữ cảnh văn bản.

- Độ dài câu văn phản ánh mức độ chi tiết của thông tin và độ phức tạp trong cấu trúc câu.
- Giá trị đầu tiên và cuối cùng của mỗi đặc trưng giúp hiểu thêm về sự biến đổi và xu hướng của các câu văn trong văn bản.
- Độ lệch chuẩn của các đặc trưng phản ánh mức độ phân tán và sự biến động trong độ dài và số lượng từ của các câu văn.

In [7]:
# Hàm xử lý câu văn
def preprocess_sentences(df):
    # Tách các câu văn từ đoạn văn bản đầy đủ và loại bỏ các câu ngắn hơn 15 ký tự
    df = df.with_columns(pl.col('full_text').map_elements(clean_text).str.split(by=".").alias("sentence"))
    df = df.explode('sentence')
    df = df.with_columns(pl.col('sentence').map_elements(lambda x: len(x)).alias("sentence_len"))
    df = df.filter(pl.col('sentence_len') >= 15)
    df = df.with_columns(pl.col('sentence').map_elements(lambda x: len(x.split(' '))).alias("sentence_word_count"))
    return df

# Hàm tính toán các đặc trưng từ câu văn
def extract_sentence_features(df):
    features = ['sentence_len', 'sentence_word_count']
    aggs = [
        *[pl.col('sentence_len').filter(pl.col('sentence_len') >= i).count().alias(f"sentence_length_ge_{i}_count") for i in [15, 50, 100, 150, 200, 250, 300]],
        *[pl.col(feat).max().alias(f"{feat}_max") for feat in features],
        *[pl.col(feat).mean().alias(f"{feat}_mean") for feat in features],
        *[pl.col(feat).min().alias(f"{feat}_min") for feat in features],
        *[pl.col(feat).first().alias(f"{feat}_first") for feat in features],
        *[pl.col(feat).last().alias(f"{feat}_last") for feat in features],
        *[pl.col(feat).sum().alias(f"{feat}_sum") for feat in features],
        *[pl.col(feat).kurtosis().alias(f"{feat}_kurtosis") for feat in features],
        *[pl.col(feat).quantile(0.25).alias(f"{feat}_q1") for feat in features],
        *[pl.col(feat).quantile(0.75).alias(f"{feat}_q3") for feat in features],
    ]
    df = df.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    return df.to_pandas()

# Xử lý dữ liệu
sentence_train = preprocess_sentences(train)
sentence_test = preprocess_sentences(test)

# Kết hợp các đặc trưng mới vào dữ liệu
train_feats = train_feats.merge(extract_sentence_features(sentence_train), on='essay_id', how='left')
test_feats = test_feats.merge(extract_sentence_features(sentence_test), on='essay_id', how='left')

# Đếm số đặc trưng
feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], train_feats.columns))
print('Feature count in train set: ', len(feature_names))
feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], test_feats.columns))
print('Feature count in test set: ', len(feature_names))

Feature count in train set:  68
Feature count in test set:  68


### Rút trích đặc trưng từ

- Độ dài từ phản ánh mức độ phức tạp, tính học thuật và mức độ thông tin của từ vựng sử dụng trong văn bản.
- Giá trị lớn nhất, trung bình, độ lệch chuẩn, và các phần tư cung cấp cái nhìn tổng quát về sự phân bố độ dài từ trong văn bản.
- Số lượng lỗi chính tả cho biết về chất lượng từ vựng và khả năng biểu đạt của văn bản.

Do số lượng từ lớn hơn rất nhiều so với câu và đoạn nên nhóm không sử dụng hàm explode vì nó có thể tạo ra một số lượng lớn các hàng mới, gây tốn bộ nhớ. Thay vào đó, xử lý từng phần nhỏ của dữ liệu,

In [8]:
# Khởi tạo SpellChecker
spellchecker = SpellChecker()

# Hàm đếm số lỗi chính tả
def count_spelling_errors(text):
    wordlist = text.split()
    amount_miss = len(list(spellchecker.unknown(wordlist)))
    return amount_miss

# Hàm xử lý từ tối ưu hóa
def preprocess_words(df):
    # Tách các từ từ đoạn văn bản đầy đủ và loại bỏ các từ có độ dài bằng 0
    df = df.with_columns(pl.col('full_text').map_elements(lambda text: text.split()).alias("words"))
    return df

# Hàm tính toán các đặc trưng từ từ tối ưu hóa
def extract_word_features(df):
    features = {
        'word_length_max': [],
        'word_length_mean': [],
        'word_length_std': [],
        'word_length_q1': [],
        'word_length_median': [],
        'word_length_q3': [],
        **{f"word_length_ge_{i+1}_count": [] for i in range(15)},
        'spelling_errors': []
    }
    essay_ids = []

    for essay_id, essay_text, essay_words in zip(df['essay_id'], df['full_text'], df['words']):
        word_lengths = [len(word) for word in essay_words if len(word) > 0]
        if not word_lengths:
            continue
        essay_ids.append(essay_id)
        features['word_length_max'].append(max(word_lengths))
        features['word_length_mean'].append(sum(word_lengths) / len(word_lengths))
        features['word_length_std'].append(pl.Series(word_lengths).std())
        features['word_length_q1'].append(pl.Series(word_lengths).quantile(0.25))
        features['word_length_median'].append(pl.Series(word_lengths).quantile(0.50))
        features['word_length_q3'].append(pl.Series(word_lengths).quantile(0.75))
        for i in range(15):
            features[f"word_length_ge_{i+1}_count"].append(sum(1 for length in word_lengths if length >= i+1))
        features['spelling_errors'].append(count_spelling_errors(essay_text))

    feature_df = pl.DataFrame({**{'essay_id': essay_ids}, **features})
    return feature_df.to_pandas()

# Xử lý dữ liệu
word_train = preprocess_words(train)
word_test = preprocess_words(test)

# Kết hợp các đặc trưng mới vào dữ liệu
train_feats = train_feats.merge(extract_word_features(word_train), on='essay_id', how='left')
test_feats = test_feats.merge(extract_word_features(word_test), on='essay_id', how='left')

# Đếm số đặc trưng
feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], train_feats.columns))
print('Feature count in train set: ', len(feature_names))
feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], test_feats.columns))
print('Feature count in test set: ', len(feature_names))


Feature count in train set:  90
Feature count in test set:  90


### Thuật toán TF-IDF (Term Frequency, Inverse Document Frequency)

"TF-IDF là viết tắt của “Term Frequency, Inverse Document Frequency” - tạm dịch “Tần suất thuật ngữ, Tần suất tài liệu nghịch đảo”. Đó là một cách để chấm điểm tầm quan trọng của các từ (hoặc \"các thuật ngữ\") dựa trên tần suất xuất hiện của chúng xuất hiện trên nhiều tài liệu dựa trên quy tắc sau:"
- Nếu một từ xuất hiện thường xuyên trong tài liệu, điều đó rất quan trọng. Cho từ này điểm cao.
- Nhưng nếu một từ xuất hiện trong nhiều tài liệu, thì đó không phải là mã định danh duy nhất. Cho từ đó điểm thấp.

Do đó, những từ phổ biến như `the` và `for` xuất hiện trong nhiều tài liệu sẽ được scaled down. Các từ xuất hiện thường xuyên trong một tài liệu sẽ được scaled up.

Với những giải thích trên, ta có công thức tính trọng số của một từ trong tài liệu trong ngữ liệu như sau:
$$w_{i,j} = tf_{i,j} \cdot idf_i = tf_{i,j} \cdot log(\frac {N}{df_i})$$

Trong đó:
- $tf_{i,j}$: Tần suất xuất hiện của i trong j
- $N$: Tổng số tài liệu
- $df_i$: Số tài liệu chứa i

**Reference:** https://medium.com/analytics-vidhya/an-introduction-to-tf-idf-using-python-5f9d1a343f77"

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import polars as pl

# Khởi tạo TfidfVectorizer với các tham số cụ thể
vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 3),
    min_df=0.05,
    max_df=0.95,
    sublinear_tf=True,
)

# Áp dụng TfidfVectorizer lên dữ liệu huấn luyện
train_tfid = vectorizer.fit_transform([i for i in train['full_text']])
test_tfid = vectorizer.transform([i for i in test['full_text']])

# Đưa kết quả vào DataFrame
tfidf_train = pd.DataFrame(train_tfid.toarray())
tfidf_test = pd.DataFrame(test_tfid.toarray())

# Đổi tên các cột
tfidf_train.columns = [f'tfid_{i}' for i in range(len(tfidf_train.columns))]
tfidf_test.columns = [f'tfid_{i}' for i in range(len(tfidf_test.columns))]
tfidf_train['essay_id'] = train_feats['essay_id']
tfidf_test['essay_id'] = test_feats['essay_id']

# Hợp nhất các đặc trưng mới với dữ liệu huấn luyện đã có
train_feats = train_feats.merge(tfidf_train, on='essay_id', how='left')
test_feats = test_feats.merge(tfidf_test, on='essay_id', how='left')

# Đếm số đặc trưng
feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], train_feats.columns))
print('Feature count in train set: ', len(feature_names))
feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], test_feats.columns))
print('Feature count in test set: ', len(feature_names))

Feature count in train set:  2918
Feature count in test set:  2918


In [10]:
# Lấy danh sách các cột của từng DataFrame
train_columns = set(train_feats.columns)
test_columns = set(test_feats.columns)

# Tìm các cột có trong train_feats nhưng không có trong test_feats
columns_only_in_train = train_columns - test_columns

print("Các cột có trong train_feats nhưng không có trong test_feats:")
print(columns_only_in_train)

Các cột có trong train_feats nhưng không có trong test_feats:
set()


## Chuẩn bị dữ liệu

### Định nghĩa thông số k-folds

In [11]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_feats = train_feats.merge(train.to_pandas(), on='essay_id', how='left')
for i, (_, val_index) in enumerate(skf.split(train_feats, train_feats['score'])):
    train_feats.loc[val_index, 'fold'] = i

train_feats.shape

(17307, 2926)

In [12]:
test_feats = test_feats.merge(test.to_pandas(), on='essay_id', how='left')

test_feats.shape

(3, 2924)

In [13]:
# Lấy danh sách các cột của từng DataFrame
train_columns = set(train_feats.columns)
test_columns = set(test_feats.columns)

# Tìm các cột có trong train_feats nhưng không có trong test_feats
columns_only_in_train = train_columns - test_columns

print("Các cột có trong train_feats nhưng không có trong test_feats:")
print(columns_only_in_train)

Các cột có trong train_feats nhưng không có trong test_feats:
{'score', 'fold'}


### Feature selection

In [14]:
target = ['score']
drop_columns = ['essay_id', 'fold', 'full_text', 'paragraph']

## Huấn luyện mô hình

### Định nghĩa hàm đánh giá và hàm mất mát

In [15]:
# Hàm tính toán QWK
def calculate_quadratic_weighted_kappa(y_true, y_pred):
    y_true_adjusted = (y_true + a).round()
    y_pred_adjusted = (y_pred + a).clip(1, 6).round()
    qwk_score = cohen_kappa_score(y_true_adjusted, y_pred_adjusted, weights="quadratic")
    return 'QWK', qwk_score, True

# Hàm mục tiêu cho QWK
def qwk_objective(y_true, y_pred):
    y_true_adjusted = y_true + a
    y_pred_adjusted = y_pred + a
    y_pred_adjusted = y_pred_adjusted.clip(1, 6)

    f = 1 / 2 * np.sum((y_pred_adjusted - y_true_adjusted) ** 2)
    g = 1 / 2 * np.sum((y_pred_adjusted - a) ** 2 + b)

    df = y_pred_adjusted - y_true_adjusted
    dg = y_pred_adjusted - a

    grad = (df / g - f * dg / g ** 2) * len(y_true_adjusted)
    hess = np.ones(len(y_true_adjusted))
    return grad, hess

# Hàm tính toán các tham số cho QWK
def calculate_qwk_parameters(y):
    mean_value = y.mean()
    variance_value = (y ** 2).mean() - mean_value ** 2
    return np.round(mean_value, 4), np.round(variance_value, 4)

In [16]:
trained_models = []

# Định nghĩa các callback cho LightGBM
training_callbacks = [
    lgb.log_evaluation(period=25),
    lgb.early_stopping(stopping_rounds=75, first_metric_only=True)
]

# Huấn luyện mô hình với K-fold Cross-Validation
for fold in range(5):

    model = lgb.LGBMRegressor(
        objective=qwk_objective, metrics='None', learning_rate=0.1, max_depth=5,
        num_leaves=10, colsample_bytree=0.5, reg_alpha=0.1, reg_lambda=0.8,
        n_estimators=1024, random_state=42, verbosity=-1
    )

    a, b = calculate_qwk_parameters(train_feats[train_feats['fold'] != fold]['score'])
    # Tách dữ liệu huấn luyện và đánh giá cho từng fold
    X_train = train_feats[train_feats['fold'] != fold].drop(columns=drop_columns+target)
    y_train = train_feats[train_feats['fold'] != fold]['score'] - a

    X_eval = train_feats[train_feats['fold'] == fold].drop(columns=drop_columns+target)
    y_eval = train_feats[train_feats['fold'] == fold]['score'] - a

    print(f'\nFold_{fold + 1} Training ================================\n')
    print(f"Fold {fold} a: {a}  ;;  b: {b}")

    # Huấn luyện mô hình
    lgb_model = model.fit(
        X_train, y_train,
        eval_names=['train', 'valid'],
        eval_set=[(X_train, y_train), (X_eval, y_eval)],
        eval_metric=calculate_quadratic_weighted_kappa,
        callbacks=training_callbacks
    )

    trained_models.append(lgb_model)



Fold_1 Training ================================

Fold 0 a: 2.9484  ;;  b: 1.0917
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.742433	valid's QWK: 0.72734
[50]	train's QWK: 0.787731	valid's QWK: 0.763251
[75]	train's QWK: 0.805905	valid's QWK: 0.775162
[100]	train's QWK: 0.820107	valid's QWK: 0.779166
[125]	train's QWK: 0.830464	valid's QWK: 0.786144
[150]	train's QWK: 0.838893	valid's QWK: 0.788155
[175]	train's QWK: 0.846064	valid's QWK: 0.790829
[200]	train's QWK: 0.85299	valid's QWK: 0.792045
[225]	train's QWK: 0.859492	valid's QWK: 0.793882
[250]	train's QWK: 0.864918	valid's QWK: 0.794021
[275]	train's QWK: 0.870287	valid's QWK: 0.794221
[300]	train's QWK: 0.874626	valid's QWK: 0.793327
[325]	train's QWK: 0.879118	valid's QWK: 0.792288
Early stopping, best iteration is:
[269]	train's QWK: 0.869182	valid's QWK: 0.795384
Evaluated only: QWK

Fold_2 Training ================================


In [17]:
# Danh sách để lưu trữ dự đoán và giá trị thực tế
predictions, actuals = [], []

# Lặp qua từng fold và mô hình đã huấn luyện
for fold, model in enumerate(trained_models):
    X_eval_fold = train_feats[train_feats['fold'] == fold].drop(columns=drop_columns+target)
    y_eval_fold = train_feats[train_feats['fold'] == fold]['score']

    # Dự đoán và điều chỉnh dự đoán với giá trị a
    fold_predictions = model.predict(X_eval_fold) + a

    actuals.extend(y_eval_fold)
    predictions.extend(np.round(fold_predictions, 0))

# Tính toán điểm Kappa trọng số bậc hai (QWK) cho các dự đoán
validation_score = cohen_kappa_score(actuals, predictions, weights="quadratic")

# In ra điểm đánh giá
print(f"Validation score: {validation_score}")

Validation score: 0.7914908666585154


In [18]:
# Danh sách để lưu trữ dự đoán từ các mô hình
test_predictions = []
drop_columns = ['essay_id', 'full_text', 'paragraph']

# Lặp qua từng mô hình để dự đoán trên tập dữ liệu kiểm thử
for fold, model in enumerate(trained_models):
    X_test = test_feats.drop(columns=drop_columns)
    fold_predictions = model.predict(X_test) + a
    test_predictions.append(fold_predictions)

# Kết hợp kết quả từ các mô hình
for i, fold_predictions in enumerate(test_predictions):
    test_feats[f"score_pred_{i}"] = fold_predictions

# Tính toán giá trị dự đoán trung bình và làm tròn kết quả để ra kết quả cuối cùng
test_feats["score"] = np.round(test_feats[[f"score_pred_{fold}" for fold in range(5)]].mean(axis=1), 0).astype('int32')

# In ra các giá trị dự đoán
test_feats[['essay_id', 'score']].head()

,essay_id,score
0,000d118,2
1,000fe60,3
2,001ab80,5
